In [2]:
# Data USA Data Pulls - Wage Disparity Section

In [3]:
#IMPORTS

In [37]:
import pandas as pd
import requests
from pprint import pprint



##### ------------------API DATA PULL ----------------------
##### --- Washington WAGE GENDER BY SOC CODE BY YEAR - 2014 thru 2016----

In [6]:
# Pull Avg Wage Data by Gender for US from Data USA JSON

WA_level_year = pd.DataFrame()

# CODE to Pull Washington Stats

year = 'all'
geo_level = "04000US53"

url = f"https://api.datausa.io/api/?sort=desc&show=soc%2Csex&required=num_ppl%2Cnum_ppl_moe%2Cavg_wage_ft%2Cavg_wage_ft_moe&sumlevel=3%2Call&year={year}&geo={geo_level}"
json = requests.get(url).json()
data = [dict(zip(json["headers"], d)) for d in json["data"]]
for d in data:
    WA_level_year = WA_level_year.append(pd.DataFrame([d]))


WA_level_year.to_csv('WA_level_year.csv', index=False)
print('done')
print(f' There are {len(WA_level_year)} records in the WA File')

done
 There are 2774 records in the WA File


### Reading in the API data to avoid re-pulling
### Reading the Washington State Data by YEAR from Data USA

In [7]:
# Read in Data
WA_data = pd.read_csv('WA_level_year.csv')

#WA_merge.info()
# Convert data to Flow and Sex to Integer
WA_data['avg_wage_ft'] = WA_data['avg_wage_ft'].astype('float64') 
WA_data['sex'] = WA_data['sex'].astype('int64')

WA_data.info()
WA_data.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2774 entries, 0 to 2773
Data columns (total 8 columns):
avg_wage_ft        2755 non-null float64
avg_wage_ft_moe    2755 non-null float64
geo                2774 non-null object
num_ppl            2774 non-null int64
num_ppl_moe        2774 non-null float64
sex                2774 non-null int64
soc                2774 non-null object
year               2774 non-null int64
dtypes: float64(3), int64(3), object(2)
memory usage: 173.5+ KB


,avg_wage_ft,avg_wage_ft_moe,geo,num_ppl,num_ppl_moe,sex,soc,year
0,111314.0,5935.55,04000US53,14386,1081.6200,1,111021,2014
1,80831.5,8495.82,04000US53,7008,776.8800,2,111021,2014
2,174698.0,27891.60,04000US53,21,24.2435,1,113111,2014
3,75822.1,13327.40,04000US53,184,104.0130,2,113111,2014
4,93550.0,11412.40,04000US53,3529,463.9780,1,113121,2014


#### MERGING Washington WAGE DATA WITH SOC NAME

In [8]:
### -----PART I ---------
### SOC NAME AND CODE FILE

In [10]:
# Import the SOC Name and SOC Code file

soc_names = pd.read_csv('SOC_NAME_SOC.csv')
soc_names.head()

# Soc file has duplicates due to where file was generated
print(f'Before Dups: {len(soc_names)}')
soc_names.drop_duplicates(subset='soc',keep='first',inplace=True )

# soc_names.head()
print(f'After Dups:  {len(soc_names)}')
soc_names['soc'].sort_values()
soc_names.tail()


Before Dups: 956
After Dups:  477


,soc_name,soc,soc_cat
946,"Subway, streetcar, & other rail transportation...",5340XX,53
948,"Sailors & marine oilers, & ship engineers",5350XX,53
950,"Miscellaneous transportation workers, includin...",5360XX,53
952,"Conveyor operators & tenders, & hoist & winch ...",5370XX,53
954,"Miscellaneous material moving workers, includi...",5371XX,53


In [11]:
### ---------------------- PART II ----------------------
### Sort the Washington State data to prep to merge with the Occupation Category Names

In [12]:
WA_data['soc'].sort_values()
WA_data.tail()

WA_merge = pd.merge(WA_data, soc_names, on="soc")

WA_merge['avg_wage_ft'] = WA_merge['avg_wage_ft'].astype('float64') 
WA_merge['sex'] = WA_merge['sex'].astype('int64')
WA_merge.head()

#WA_merge.info()

,avg_wage_ft,avg_wage_ft_moe,geo,num_ppl,num_ppl_moe,sex,soc,year,soc_name,soc_cat
0,111314.0,5935.55,04000US53,14386,1081.620,1,111021,2014,General & operations managers,11
1,80831.5,8495.82,04000US53,7008,776.880,2,111021,2014,General & operations managers,11
2,109514.0,6349.89,04000US53,15088,988.957,1,111021,2015,General & operations managers,11
3,80114.1,9167.05,04000US53,6679,709.046,2,111021,2015,General & operations managers,11
4,111536.0,6295.11,04000US53,15561,1051.800,1,111021,2016,General & operations managers,11


#### GENERATE THE WAGE DIFFERENCE AND WAGE DIFFERENCE PERCENT FIELDS IN Washingon LEVEL DATA

In [13]:
WA_data.head()

#Pull the first 2 digits of the SOC to evaluate Wage Difference at the Category Level of Wages
#Identified in above analyses that the merge of SOC Codes caused categories of occupations to be missed in the merge
WA_data['soc_cat'] = WA_data['soc'].str[0:2]

reduced_WA_merge = WA_data[["soc_cat","soc", "sex","num_ppl", "avg_wage_ft", "year"]]
reduced_WA_merge.head()

# Split the Male and Female data sets to rename the fields
WA_male = reduced_WA_merge.loc[reduced_WA_merge['sex'] == 1]
WA_male = WA_male.rename(columns = {"num_ppl" : "m_num_ppl", "avg_wage_ft" : "m_avg_wage_ft"})

WA_female = reduced_WA_merge.loc[reduced_WA_merge['sex'] == 2]
WA_female = WA_female.rename(columns = {"num_ppl" : "f_num_ppl", "avg_wage_ft" : "f_avg_wage_ft"})

WA_female.head()

# Merge back the male and female to evaluate the occupation and wage differences
WA_mf_merge = pd.merge(WA_female, WA_male, on=['soc_cat', 'soc', 'year'])

WA_mf_merge
# # QA data here:  https://datausa.io/profile/geo/united-states/

# TOTAL THE WAGES ACROSS MEN AND WOMEN 
WA_mf_merge['m_wage_total'] = WA_mf_merge['m_avg_wage_ft'] * WA_mf_merge['m_num_ppl']
WA_mf_merge['f_wage_total'] = WA_mf_merge['f_avg_wage_ft'] * WA_mf_merge['f_num_ppl']

# WA_mf_merge.loc[(WA_mf_merge.m_num_ppl+ WA_mf_merge.f_num_ppl) > 5000].head(10)

del WA_mf_merge['sex_x']
del WA_mf_merge['sex_y']
del WA_mf_merge['f_avg_wage_ft']
del WA_mf_merge['m_avg_wage_ft']

WA_mf_merge.head(5)


,soc_cat,soc,f_num_ppl,year,m_num_ppl,m_wage_total,f_wage_total
0,11,111021,7008,2014,14386,1.601363e+09,566467152.0
1,11,113111,184,2014,21,3.668658e+06,13951266.4
2,11,113121,5202,2014,3529,3.301380e+08,426848029.2
3,11,113131,575,2014,779,8.329691e+07,40497480.0
4,11,119161,91,2014,297,2.292977e+07,10841376.0


#### VIEW OF TECH SOC CODE AND NAMES ONLY

In [14]:
computer = WA_mf_merge.loc[WA_mf_merge.soc_cat == '15']
computer.head()

,soc_cat,soc,f_num_ppl,year,m_num_ppl,m_wage_total,f_wage_total
33,15,151111,124,2014,295,41264895.0,12581536.0
34,15,151121,4657,2014,7182,686981282.4,343971142.7
35,15,151122,244,2014,1042,97269449.6,20031887.6
36,15,151131,2132,2014,9558,930839283.0,162507862.4
37,15,151134,2152,2014,3461,261766505.2,129379531.2


In [15]:
WA_mf_merge.describe()
WA_mf_merge.columns

Index(['soc_cat', 'soc', 'f_num_ppl', 'year', 'm_num_ppl', 'm_wage_total',
       'f_wage_total'],
      dtype='object')

#####  IDENTIFY % OF TOTAL TECH JOBS IN Washingong
#####   IDENTIFY AVG WAGES AND AVG WAGE DIFFERENCES IN TECH IN Washington
##### IDENTIFY THE TOP JOB CATEGORY FOR AVG WAGE DIFFERENCES IN Washington

In [16]:
#list columns to remove extraneous
WA_mf_merge.columns

# Group the fields by category code
soc_cat = WA_mf_merge.groupby(['soc_cat','year']).sum()

soc_cat

# Average WAGES FEMALE AND MALE
soc_cat['m_avg_wage'] = soc_cat['m_wage_total'] / soc_cat['m_num_ppl']
soc_cat['f_avg_wage'] = soc_cat['f_wage_total'] / soc_cat['f_num_ppl']

# Sum all of the people
soc_cat['tot_cat_ppl'] = soc_cat['f_num_ppl'] + soc_cat['m_num_ppl']


# WAGE DIFFERENCE
soc_cat['wage_diff'] = soc_cat['m_avg_wage'] - soc_cat['f_avg_wage']

# PCT WAGE DIFFERENCE
soc_cat['pct_wage_diff'] = soc_cat.wage_diff / soc_cat.m_avg_wage

# PCT WOMEN
soc_cat['pct_female'] = soc_cat.f_num_ppl / (soc_cat.f_num_ppl + soc_cat.m_num_ppl)


# AVG WAGE 
soc_cat['avg_wage'] = (soc_cat.m_wage_total + soc_cat.f_wage_total) / (soc_cat.f_num_ppl + soc_cat.m_num_ppl)
soc_cat.head()

soc_cat.reset_index(inplace=True)
soc_cat.set_index('year',inplace=True)

soc_cat.head()
# #mycat11 = soc_cat.loc[soc_cat.soc_cat == 11&15&41]
#OR
#mycat = soc_cat.loc[(soc_cat['soc_cat'] == '11') | (soc_cat['soc_cat'] == '15') | (soc_cat['soc_cat'] =='41')]
#AND
#mycat11 = soc_cat.loc[(soc_cat['soc_cat'] == 11) & (soc_cat['soc_cat'] ==15)]

#mycat
# #mycat15 = soc_cat.loc[soc_cat.soc_cat == 15]
# #mycat41 = soc_cat.loc[soc_cat.soc_cat == 41]




,soc_cat,f_num_ppl,m_num_ppl,m_wage_total,f_wage_total,m_avg_wage,f_avg_wage,tot_cat_ppl,wage_diff,pct_wage_diff,pct_female,avg_wage
year,,,,,,,,,,,,
2014,11,129806,193182,2.007175e+10,9.730757e+09,103900.737471,74963.845381,322988,28936.892091,0.278505,0.401891,92271.258312
2015,11,135371,200469,2.115702e+10,1.022517e+10,105537.591580,75534.383631,335840,30003.207949,0.284289,0.403082,93443.843774
2016,11,140286,208714,2.268321e+10,1.079414e+10,108680.843843,76943.828351,349000,31737.015492,0.292020,0.401966,95923.654859
2014,13,84062,65864,5.708204e+09,5.430023e+09,86666.531497,64595.453933,149926,22071.077564,0.254667,0.560690,74291.500333
2015,13,86421,68507,5.938485e+09,5.533041e+09,86684.351548,64024.270604,154928,22660.080944,0.261409,0.557814,74044.242238


#### MERGE with higher level Soc Cat and SOC Occupation Names
### Send merge_categories to CSV for Bubble Chart of 2016 Occupation Categories

In [17]:
## Read in the SOC_Category_Name File
categories = pd.read_csv('SOC_Category_Name.csv')
categories.head()
soc_cat['soc_cat'] = soc_cat['soc_cat'].astype(int)

soc_cat.reset_index(inplace=True)
data2016 = soc_cat[soc_cat['year'] == 2016]
merge_categories = pd.merge(data2016, categories, on=('soc_cat'), how='inner')

#del merge_categories.m_wage_total
#del merge_categories.f_wage_total

# Define index to Category name
#merge_categories.set_index('category_name')

merge_categories.to_csv('merge_categories2016.csv')

In [18]:
#revisit WA_head to identify the pct_wage_disparity avg in QA
WA_data['total_wage'] = WA_data.num_ppl * WA_data.avg_wage_ft

WA_totals = WA_data.groupby(['year','sex']).sum()

WA_totals['Avg_Wage'] = WA_totals['total_wage'] / WA_totals['num_ppl']

pd.options.display.float_format = '${:,.2f}'.format

#Identify the Wage Disparity Across all Occupation Categories in 2016
# 28.7% 29% rounded up
WA_totals


avg_wage_ft  avg_wage_ft_moe  num_ppl  num_ppl_moe  \
year sex                                                         
2014 1   $28,603,037.80    $4,918,959.35  1648195  $202,080.18   
     2   $22,322,961.10    $4,124,543.66  1418804  $162,497.03   
2015 1   $28,583,775.78    $4,956,964.72  1692147  $205,237.54   
     2   $22,477,205.34    $4,162,789.69  1444491  $164,494.61   
2016 1   $29,223,416.72    $5,153,846.39  1729328  $205,758.38   
     2   $22,661,444.79    $4,130,748.16  1474163  $165,310.80   

                  total_wage   Avg_Wage  
year sex                                 
2014 1   $106,528,347,200.70 $64,633.34  
     2    $65,692,249,314.33 $46,301.14  
2015 1   $110,494,706,555.64 $65,298.53  
     2    $67,555,640,562.26 $46,767.78  
2016 1   $116,100,049,986.96 $67,135.93  
     2    $70,602,321,275.38 $47,893.16

### Look at Tech Occupation Categories:  11,15, 41 by 2014 thru 2016

In [19]:
soc_cat.head()
tech_1416 = soc_cat[(soc_cat['soc_cat'] == 11) | (soc_cat['soc_cat'] == 15) | (soc_cat['soc_cat'] == 41)]
tech_1416

,year,soc_cat,f_num_ppl,m_num_ppl,m_wage_total,f_wage_total,m_avg_wage,f_avg_wage,tot_cat_ppl,wage_diff,pct_wage_diff,pct_female,avg_wage
0,2014,11,129806,193182,"$20,071,752,266.20","$9,730,756,913.50","$103,900.74","$74,963.85",322988,"$28,936.89",$0.28,$0.40,"$92,271.26"
1,2015,11,135371,200469,"$21,157,015,446.40","$10,225,165,046.50","$105,537.59","$75,534.38",335840,"$30,003.21",$0.28,$0.40,"$93,443.84"
2,2016,11,140286,208714,"$22,683,213,641.80","$10,794,141,904.00","$108,680.84","$76,943.83",349000,"$31,737.02",$0.29,$0.40,"$95,923.65"
6,2014,15,29169,96575,"$9,754,793,025.80","$2,325,436,959.70","$101,007.43","$79,722.89",125744,"$21,284.55",$0.21,$0.23,"$96,070.03"
7,2015,15,29353,102576,"$10,609,055,397.80","$2,438,911,363.40","$103,426.29","$83,089.00",131929,"$20,337.29",$0.20,$0.22,"$98,901.43"
8,2016,15,30216,108501,"$11,671,192,653.60","$2,541,498,550.70","$107,567.60","$84,111.02",138717,"$23,456.59",$0.22,$0.22,"$102,458.18"
45,2014,41,148690,157406,"$10,330,696,955.70","$6,053,832,504.60","$65,630.90","$40,714.46",306096,"$24,916.44",$0.38,$0.49,"$53,527.42"
46,2015,41,151001,157530,"$10,528,256,470.60","$6,162,862,884.60","$66,833.34","$40,813.39",308531,"$26,019.95",$0.39,$0.49,"$54,098.68"
47,2016,41,150931,156369,"$10,836,736,276.30","$6,430,760,856.40","$69,302.33","$42,607.29",307300,"$26,695.04",$0.39,$0.49,"$56,191.01"


In [20]:
#Send the 3 Year file of Tech to Data Visualization Section for 3-Yr Bar Chart
tech_1416.to_csv('tech_1416.csv')

In [21]:
## Pull the US LEVEL DATA OF OCCUPATIONS TO COMPARE AGAINST WA TECH OCCCUPATIONS

In [20]:
# COUNTY FIF CODE AND STATE 

county_fif = pd.read_csv('State-County-FIFS.txt', header=None ,dtype=str, names=['state_cd', 'f1','f2', 'county', 'hlevel'])
county_fif.head(7)
county_fif['state_fif_code'] = '04000US' + county_fif['f1']

print(len(county_fif))
county_fif[county_fif['state_cd']=='DC']

# FILE IS AT COUNTY LEVEL - NEED STATE LEVEL CODES
# BEFORE DUPES REMOVED
print(f'Before Dups: {len(county_fif)}')
county_fif.drop_duplicates(subset='state_fif_code',keep='first',inplace=True )

# AFTER DUPES REMOVED
print(f'After Dups:  {len(county_fif)}')
county_fif.tail()

3143
Before Dups: 3143
After Dups:  51


,state_cd,f1,f2,county,hlevel,state_fif_code
2820,VA,51,001,Accomack County,H1,04000US51
2954,WA,53,001,Adams County,H1,04000US53
2993,WV,54,001,Barbour County,H1,04000US54
3048,WI,55,001,Adams County,H1,04000US55
3120,WY,56,001,Albany County,H1,04000US56


In [ ]:
# Pull USA Data by State

In [21]:
# PullState level OCCUPATIONS

state_level = pd.DataFrame()
for state in county_fif['state_fif_code']:
    url = f"https://api.datausa.io/api/?sort=desc&show=soc%2Csex&required=num_ppl%2Cnum_ppl_moe%2Cavg_wage_ft%2Cavg_wage_ft_moe&sumlevel=3%2Call&year=latest&geo={state}"
    json = requests.get(url).json()
    print(f'data being pulled by {state}')
    data = [dict(zip(json["headers"], d)) for d in json["data"]]
    for d in data:
        state_level = state_level.append(pd.DataFrame([d]))


print('done')

len(state_level)

data being pulled by 04000US01
data being pulled by 04000US02
data being pulled by 04000US04
data being pulled by 04000US05
data being pulled by 04000US06
data being pulled by 04000US08
data being pulled by 04000US09
data being pulled by 04000US10
data being pulled by 04000US11
data being pulled by 04000US12
data being pulled by 04000US13
data being pulled by 04000US15
data being pulled by 04000US16
data being pulled by 04000US17
data being pulled by 04000US18
data being pulled by 04000US19
data being pulled by 04000US20
data being pulled by 04000US21
data being pulled by 04000US22
data being pulled by 04000US23
data being pulled by 04000US24
data being pulled by 04000US25
data being pulled by 04000US26
data being pulled by 04000US27
data being pulled by 04000US28
data being pulled by 04000US29
data being pulled by 04000US30
data being pulled by 04000US31
data being pulled by 04000US32
data being pulled by 04000US33
data being pulled by 04000US34
data being pulled by 04000US35
data bei

44560

In [22]:
state_level.to_csv('State_Level.csv',index=False)

state_level.head()
state_level.columns

Index(['avg_wage_ft', 'avg_wage_ft_moe', 'geo', 'num_ppl', 'num_ppl_moe',
       'sex', 'soc', 'year'],
      dtype='object')

In [38]:
# Ready State level data back in as csv file
data = pd.read_csv('State_Level.csv', index_col='geo')

#Keep and reorder the fields
state_data = data[['soc', 'num_ppl','sex','avg_wage_ft']]

state_data.reset_index(inplace=True)
state_data['f1'] = state_data['geo'].astype(str).str[7:9]
state_data['soc_cat'] = state_data['soc'].astype(str).str[0:2]
state_data

state_tech = state_data[(state_data['soc_cat'] =='11') |
                        (state_data['soc_cat'] =='15') |
                        (state_data['soc_cat'] =='41')]
        

state_tech['soc_cat'].value_counts()
#state_cd = state_data.loc[state_data.geo]


D:\RUTGERS\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


11    2683
41    1792
15    1381
Name: soc_cat, dtype: int64

In [39]:
# MERGING THE FIF WITH THE STATE FILE
county_fif.head()
states = county_fif[['state_cd', 'f1']]

all_states = pd.merge(state_tech, states, on='f1')

all_states['state_cd'].value_counts()

all_states.head()

,geo,soc,num_ppl,sex,avg_wage_ft,f1,soc_cat,state_cd
0,04000US01,111021,7650,1,98856.6,01,11,AL
1,04000US01,111021,2913,2,56630.0,01,11,AL
2,04000US01,1110XX,9786,1,152922.0,01,11,AL
3,04000US01,1110XX,2892,2,112979.0,01,11,AL
4,04000US01,112011,99,1,60580.6,01,11,AL


In [40]:
## Importing the CSV File to avoid a re-pull of the API data

In [41]:
all_states.columns

#Drop unneeded Fields
state = all_states[['state_cd', 'soc_cat','num_ppl','sex','avg_wage_ft','f1']]

state['total_wages'] = state.avg_wage_ft * state.num_ppl

reduced_state = state.groupby(['state_cd','f1','soc_cat','sex']).sum()
reduced_state.head(10)

D:\RUTGERS\Anaconda2\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


num_ppl  avg_wage_ft   total_wages
state_cd f1 soc_cat sex                                    
AK       02 11      1      19411    2128423.0  1.934556e+09
                    2      13860    1829299.1  9.617051e+08
            15      1       5199     901502.5  3.600911e+08
                    2        998     717193.1  6.223249e+07
            41      1      13037    1035216.9  8.023149e+08
                    2      14648     547625.0  5.058758e+08
AL       01 11      1     100194    2281910.7  9.678064e+09
                    2      67586    1641514.2  3.970325e+09
            15      1      30594    1101293.2  2.447380e+09
                    2      11876     946199.1  7.593396e+08

In [43]:
#reduced_state.reset_index(inplace=True)
reduced_state.head()
reduced_state.reset_index(inplace=True)


# Split the Male and Female data sets to rename the fields
state_male = reduced_state.loc[reduced_state['sex'] == 1]
state_male = state_male.rename(columns = {"num_ppl" : "m_num_ppl", "total_wages" : "m_total_wages"})

state_female = reduced_state.loc[reduced_state['sex'] == 2]
state_female = state_female.rename(columns = {"num_ppl" : "f_num_ppl", "total_wages" : "f_total_wages"})

state_female.head()

# Merge back the male and female to evaluate the occupation and wage differences
state_mf_merge = pd.merge(state_female, state_male, on=["state_cd",'f1', 'soc_cat'])

# AVERAGE WAGES
state_mf_merge['f_avg_wage'] = state_mf_merge['f_total_wages'] / state_mf_merge['f_num_ppl']
state_mf_merge['m_avg_wage'] = state_mf_merge['m_total_wages'] / state_mf_merge['m_num_ppl']

#WAGE DIFFERENCE
state_mf_merge['wage_diff'] = state_mf_merge['m_avg_wage'] - state_mf_merge['f_avg_wage']

#PCT WAGE DIFFERENCE
state_mf_merge['pct_wage_diff'] = state_mf_merge['wage_diff'] / state_mf_merge['m_avg_wage']

# CALCULATE % OF WOMEN
state_mf_merge['perc_women'] = state_mf_merge['f_num_ppl'] / (state_mf_merge['f_num_ppl'] + state_mf_merge['m_num_ppl'])

#state_mf_merge.info()
state_mf_merge.head()

state_mf_merge.sort_values(by=['pct_wage_diff'], ascending=False, inplace=True)

del state_mf_merge['sex_x']
del state_mf_merge['avg_wage_ft_x']
del state_mf_merge['sex_y']
del state_mf_merge['avg_wage_ft_y']

state_mf_merge.loc[state_mf_merge['state_cd'] == 'WA' ]

,state_cd,f1,soc_cat,f_num_ppl,f_total_wages,m_num_ppl,m_total_wages,f_avg_wage,m_avg_wage,wage_diff,pct_wage_diff,perc_women
143,WA,53,41,150931,6.430761e+09,156369,1.083674e+10,42607.289797,69302.331513,26695.041715,0.385197,0.491152
141,WA,53,11,140286,1.079414e+10,208714,2.268321e+10,76943.828351,108680.843843,31737.015492,0.292020,0.401966
142,WA,53,15,30216,2.541499e+09,108501,1.167119e+10,84111.019020,107567.604479,23456.585459,0.218064,0.217825


In [46]:

# SENDING FILES TO CSV TO PICK UP FOR GRAPHING
state_tech_occ11 = state_mf_merge.loc[state_mf_merge['soc_cat'] == '11']
state_tech_occ15 = state_mf_merge.loc[state_mf_merge['soc_cat'] == '15']
state_tech_occ41 = state_mf_merge.loc[state_mf_merge['soc_cat'] == '41']

state_tech_occ11.to_csv('state_tech_occ11.csv')
state_tech_occ15.to_csv('state_tech_occ15.csv')
state_tech_occ41.to_csv('state_tech_occ4.csv')

state_tech_occ41

,state_cd,f1,soc_cat,f_num_ppl,f_total_wages,m_num_ppl,m_total_wages,f_avg_wage,m_avg_wage,wage_diff,pct_wage_diff,perc_women
56,LA,22,41,116490,3.333562e+09,88267,5.489164e+09,28616.725036,62188.178684,33571.453648,0.539837,0.568918
20,CT,09,41,85686,4.043714e+09,92937,8.774727e+09,47192.232225,94415.865718,47223.633493,0.500166,0.479703
149,WV,54,41,38346,9.770647e+08,33934,1.726004e+09,25480.224391,50863.552478,25383.328087,0.499047,0.530520
152,WY,56,41,12959,3.648479e+08,11550,6.482187e+08,28154.019855,56122.832476,27968.812621,0.498350,0.528745
5,AL,01,41,115107,3.448079e+09,90565,5.243885e+09,29955.425782,57901.899078,27946.473296,0.482652,0.559663
77,MS,28,41,70772,1.953066e+09,52196,2.779556e+09,27596.587529,53252.273211,25655.685682,0.481776,0.575532
32,GA,13,41,255931,9.258203e+09,233135,1.622437e+10,36174.604483,69592.178012,33417.573529,0.480192,0.523306
86,ND,38,41,17075,5.859586e+08,18763,1.232495e+09,34316.758032,65687.509711,31370.751678,0.477576,0.476450
38,IA,19,41,71334,2.296725e+09,74850,4.606592e+09,32196.775353,61544.317193,29347.541840,0.476852,0.487974
140,VT,50,41,15842,5.030668e+08,13179,7.967058e+08,31755.256956,60452.675742,28697.418786,0.474709,0.545881
